# Installing Libs

In [ ]:
!pip install tensorflow-text
!pip install tensorflow_hub==0.16.1
!pip install tf-keras
!pip install transformers
!pip install -U "tensorflow-text==2.15.*"
!pip install -U "tf-models-official==2.15.*"
!pip install tf-models-official
!pip install seaborn
!pip install tensorflow-models
!pip install gradio

In [ ]:
import os
import shutil
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, Dense, Dropout, Activation, Embedding
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.stats import spearmanr
import seaborn as sns
from math import floor, ceil
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from google.colab import drive
import gradio as gr

In [ ]:
from official.nlp import optimization

# Read dataset

In [ ]:
drive.mount('/gdrive')

In [ ]:
nRowsRead = None
df_org = pd.read_csv('/gdrive/MyDrive/CS431/dataset/labeled_data.csv', delimiter=',', nrows = nRowsRead)
df_eda = pd.read_csv('/gdrive/MyDrive/CS431/dataset/augmented_data.csv', delimiter=',', nrows = nRowsRead)
# df.dataframeName = 'labeled_data.csv'

# Preprocessing

In [ ]:
nRow, nCol = df_org.shape
print('Original dataset:  {} rows and {} columns'.format(nRow, nCol))
nRow1, nCol1 = df_eda.shape
print('EDA dataset: {} rows and {} columns'.format(nRow1, nCol1))

In [ ]:
# Doing Transformation
c = df_org['class']
df_org.rename(columns={'tweet' : 'text',
                   'class' : 'category'},
                    inplace=True)
a = df_org['text']
b =df_org['category'].map({0: 'hate_speech', 1: 'offensive_language',2: 'neither'})
df_org= pd.concat([a,b,c], axis=1)
df_org.rename(columns={'class' : 'label'},
                    inplace=True)

hate, offensive, neither = np.bincount(df_org['label'])
total = hate + offensive + neither
print('Examples:\n    Total: {}\n    Hate: {} ({:.2f}% of total)\n'.format(
    total, hate, 100 * hate / total))
print('Examples:\n    Total: {}\n    Offensive: {} ({:.2f}% of total)\n'.format(
    total, offensive, 100 * offensive / total))
print('Examples:\n    Total: {}\n    Neither: {} ({:.2f}% of total)\n'.format(
    total, neither, 100 * neither / total))


In [ ]:
# On the EDA dataset
# Doing Transformation
c = df_eda['class']
df_eda.rename(columns={'tweet' : 'text',
                   'class' : 'category'},
                    inplace=True)
a = df_eda['text']
b =df_eda['category'].map({0: 'hate_speech', 1: 'offensive_language',2: 'neither'})
df_eda= pd.concat([a,b,c], axis=1)
df_eda.rename(columns={'class' : 'label'},
                    inplace=True)

hate, offensive, neither = np.bincount(df_eda['label'])
total = hate + offensive + neither
print('Examples:\n    Total: {}\n    Hate: {} ({:.2f}% of total)\n'.format(
    total, hate, 100 * hate / total))
print('Examples:\n    Total: {}\n    Offensive: {} ({:.2f}% of total)\n'.format(
    total, offensive, 100 * offensive / total))
print('Examples:\n    Total: {}\n    Neither: {} ({:.2f}% of total)\n'.format(
    total, neither, 100 * neither / total))

In [ ]:
df = df_eda

In [ ]:
import re
import string
import nltk

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
stopwords.add("RT")
stopwords.add("Name")
print(stopwords)

In [ ]:
stemmer = nltk.SnowballStemmer("english")

In [ ]:
def clean(text):

    # Convert text to lowercase
    text = str(text).lower()

    # Remove square brackets and their contents
    # text = re.sub('\[.*?\]', '', text)

    # Remove URLs
    text = re.sub('https?://\S+|www\.\S+', '', text)

    # Remove HTML tags
    text = re.sub('<.*?>+', '', text)

    # Remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)

    # Remove newline characters
    text = re.sub('\n', '', text)

    # Remove words containing digits
    text = re.sub('\w*\d\w*', '', text)

    # Remove stopwords
    text = [word for word in text.split(' ') if word not in stopwords]
    text = " ".join(text)

    # Apply stemming
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

In [ ]:
df["text"] = df["text"].apply(clean)

# Splitting the dataset

In [ ]:
# Splitting Dataset
X_train_, X_test, y_train_, y_test = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.10,
    random_state=42,
    stratify=df.label.values,
)

X_train, X_val, y_train, y_val = train_test_split(
    df.loc[X_train_].index.values,
    df.loc[X_train_].label.values,
    test_size=0.10,
    random_state=42,
    stratify=df.loc[X_train_].label.values,
)

df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
df.loc[X_test, 'data_type'] = 'test'

df_train = df.loc[df["data_type"]=="train"]
df_val = df.loc[df["data_type"]=="val"]
df_test = df.loc[df["data_type"]=="test"]


# Printing Wordclouds

In [ ]:
import random
def random_color_func(word=None, font_size=None, position=None,  orientation=None, font_path=None, random_state=None):
    h = 344
    s = int(100.0 * 255.0 / 255.0)
    l = int(100.0 * float(random_state.randint(60, 120)) / 255.0)
    return "hsl({}, {}%, {}%)".format(h, s, l)

wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=200,
                          max_font_size=60,
                          random_state=42
                         ).generate(str(df.loc[df["category"]=="offensive_language"].text))
print("Wordcloud for offensive_language class: ")
fig = plt.figure(1)
plt.imshow(wordcloud.recolor(color_func= random_color_func, random_state=42),
           interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
def random_color_func(word=None, font_size=None, position=None,  orientation=None, font_path=None, random_state=None):
    h = 180
    s = int(100.0 * 255.0 / 255.0)
    l = int(100.0 * float(random_state.randint(60, 120)) / 255.0)
    return "hsl({}, {}%, {}%)".format(h, s, l)

wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=200,
                          max_font_size=60,
                          random_state=42
                         ).generate(str((df.loc[df["category"]=="hate_speech"].text)))
print("Wordcloud for hate_speech class: ")
fig = plt.figure(1)
plt.imshow(wordcloud.recolor(color_func= random_color_func, random_state=42),
           interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
def random_color_func(word=None, font_size=None, position=None,  orientation=None, font_path=None, random_state=None):
    h = 20
    s = int(100.0 * 255.0 / 255.0)
    l = int(100.0 * float(random_state.randint(60, 120)) / 255.0)
    return "hsl({}, {}%, {}%)".format(h, s, l)

wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=200,
                          max_font_size=60,
                          random_state=42
                         ).generate(str((df.loc[df["category"]=="neither"].text)))
print("Wordcloud for neither class: ")
fig = plt.figure(1)
plt.imshow(wordcloud.recolor(color_func= random_color_func, random_state=42),
           interpolation="bilinear")
plt.axis('off')
plt.show()

#Build model

In [ ]:
# Build input:
# Creating tensors
train_ds = tf.data.Dataset.from_tensor_slices((df_train.text.values, df_train.label.values))
val_ds = tf.data.Dataset.from_tensor_slices((df_val.text.values, df_val.label.values))
test_ds = tf.data.Dataset.from_tensor_slices((df_test.text.values, df_test.label.values))

train_ds = train_ds.shuffle(len(df_train)).batch(32, drop_remainder=False)
val_ds = val_ds.shuffle(len(df_val)).batch(32, drop_remainder=False)
test_ds = test_ds.shuffle(len(df_test)).batch(32, drop_remainder=False)

In [ ]:
bert_model_name = 'small_bert/bert_en_uncased_L-2_H-128_A-2'

In [ ]:
map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

In [ ]:
tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]
print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [ ]:
# Unbalanced dataset -> Need to define classweight to give into model
weight_for_0 = (1 / hate)*(total)/3.0
weight_for_1 = (1 / offensive)*(total)/3.0
weight_for_2 = (1 / neither)*(total)/3.0

# Defining classweight to give into the model
class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))

# BERT

In [ ]:
def BERT_model():
    # Input layer for raw text
    text_in = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

    # Preprocessing layer using the BERT model (tokenization)
    preprocess_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_in = preprocess_layer(text_in)

    # BERT encoder layer
    encode = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encode(encoder_in)

    # Extract pooled output from BERT
    pooled_output = outputs['pooled_output']  # Shape: (batch_size, 768)

    # Dropout layer for regularization
    dropout = tf.keras.layers.Dropout(0.1)(pooled_output)

    # Dense layer for final classification
    classifier = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(dropout)

    # Create and return the model
    model = tf.keras.Model(inputs=text_in, outputs=classifier)
    return model

In [ ]:
# Building model
bert_model = BERT_model()

# Model summary
bert_model.summary()


In [ ]:
# Defining loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Fine-tuning and Training models
epochs_count = 20
steps_each_epoch = tf.data.experimental.cardinality(train_ds).numpy()
number_of_training_steps = steps_each_epoch * epochs_count
number_of_warmup_steps = int(0.1 * number_of_training_steps)
init_lr = 2e-5

# Creating optimizer
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=number_of_training_steps,
                                          num_warmup_steps=number_of_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
# Model compiling
bert_model.compile(optimizer=optimizer,
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

print(f'Training model with {tfhub_handle_encoder}')

In [ ]:
# Training model
bert_history = bert_model.fit(train_ds,
                              validation_data=val_ds,
                              epochs=epochs_count,
                              class_weight=class_weight)

In [ ]:
save_dir = '/gdrive/MyDrive/CS431/model'

In [ ]:
# Saving BERT model
bert_model.save(f"{save_dir}/bert_model.h5")
print("Model saved as 'bert_model.h5' in", save_dir)

In [ ]:
 # Creating graphs for accuracy and loss
 # Fining values
history_dict = bert_history.history
print(history_dict.keys())
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']
epochs = range(1, len(acc) + 1)

# Defining figures
fig = plt.figure(figsize=(12, 10))
fig.tight_layout()
# 1st figure
plt.subplot(2, 1, 1)
plt.plot(epochs, loss, 'r', label='Training-loss')
plt.plot(epochs, val_loss, 'b', label='Validation-loss')
plt.title('Training & validation Loss')
plt.xlabel('Epochs-Count')
plt.ylabel('Loss')
plt.legend()
# 2nd Figure
plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training-accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation-accuracy')
plt.title('Training & Validation Accuracy')
plt.xlabel('Epochs-Count')
plt.ylabel('Acc')
plt.legend(loc='lower right')

In [ ]:
model_path = os.path.join(save_dir, 'bert_model.h5')
bert_model.save(model_path)
print(f"Model saved to {model_path}")

In [ ]:
# Predicting on test dataset
text = []
test_labels = []
predictions = []
for text, label in test_ds.take(-1):
  text = text.numpy()
  test_labels.append(label.numpy())
  predictions.append(bert_model.predict(text))

from itertools import chain
flatten_list = list(chain.from_iterable(predictions))

y_pred = np.argmax(flatten_list, axis=-1)
y_test = np.array(list(chain.from_iterable(test_labels)))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

# Drawing Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1', 'Class 2'], yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

## BERT + LSTM

In [ ]:
def BERT_LSTM_model():
    # Input layer for raw text
    text_in = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

    # Preprocessing layer using the BERT model (tokenization)
    preprocess_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_in = preprocess_layer(text_in)

    # BERT encoder layer
    encode = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encode(encoder_in)

    # Extract sequence output from BERT
    sequence_output = outputs['sequence_output']  # Shape: (batch_size, seq_length, 768)

    # First LSTM layer
    network_layer = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.0)
    )(sequence_output)

    # First Dropout after LSTM
    network_layer = tf.keras.layers.Dropout(0.1)(network_layer)

    # Second LSTM layer
    network_layer = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.0)
    )(network_layer)

    # Second Dropout after LSTM
    network_layer = tf.keras.layers.Dropout(0.1)(network_layer)

    # Dense layer for final classification
    network_layer = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(network_layer)

    # Create and return the model
    model = tf.keras.Model(inputs=text_in, outputs=network_layer)
    return model

In [ ]:
 # Building model
lstm_model = BERT_LSTM_model()
 # Defining loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
lstm_model.summary()

In [ ]:
from official.nlp import optimization

In [ ]:
# Fine-tuning and Training models
epochs_count = 20
steps_each_epoch = tf.data.experimental.cardinality(train_ds).numpy()
number_of_training_steps = steps_each_epoch * epochs_count
number_of_warmup_steps = int(0.1*number_of_training_steps)
init_lr = 2e-5

# Creating_optimiser
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=number_of_training_steps,
                                          num_warmup_steps=number_of_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
# Model Compiling
lstm_model.compile(optimizer=optimizer,
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=tf.keras.metrics.SparseCategoricalAccuracy('accuracy'))

print(f'Training model with {tfhub_handle_encoder}')

In [ ]:
 # Training Model
lstm_history = lstm_model.fit(x=train_ds,
                             validation_data=val_ds,
                             epochs=epochs_count,
                             class_weight=class_weight)

In [ ]:
# Lưu mô hình LSTM
lstm_model.save(f"{save_dir}/bert_lstm_model.h5")
print("Model saved as 'bert_lstm_model.h5' in", save_dir)

In [ ]:
model_path = os.path.join(save_dir, 'bert_lstm_model.h5')
lstm_model.save(model_path)
print(f"Model saved to {model_path}")

In [ ]:
 # Creating graphs for accuracy and loss
 # Fining values
history_dict = lstm_history.history
print(history_dict.keys())
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']
epochs = range(1, len(acc) + 1)

# Defining figures
fig = plt.figure(figsize=(12, 10))
fig.tight_layout()
# 1st figure
plt.subplot(2, 1, 1)
plt.plot(epochs, loss, 'r', label='Training-loss')
plt.plot(epochs, val_loss, 'b', label='Validation-loss')
plt.title('Training & validation Loss')
plt.xlabel('Epochs-Count')
plt.ylabel('Loss')
plt.legend()
# 2nd Figure
plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training-accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation-accuracy')
plt.title('Training & Validation Accuracy')
plt.xlabel('Epochs-Count')
plt.ylabel('Acc')
plt.legend(loc='lower right')

In [ ]:
# Predicting on test dataset
text = []
test_labels = []
predictions = []
for text, label in test_ds.take(-1):
  text = text.numpy()
  test_labels.append(label.numpy())
  predictions.append(lstm_model.predict(text))

from itertools import chain
flatten_list = list(chain.from_iterable(predictions))

y_pred = np.argmax(flatten_list, axis=-1)
y_test = np.array(list(chain.from_iterable(test_labels)))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

# Drawing Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1', 'Class 2'], yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
def predict_lstm(text):
    prediction = lstm_model.predict([text])
    predicted_class = np.argmax(prediction, axis=-1)
#     if predicted_class == '0':
#       return 'Hate'
#     elif predicted_class == '1':
#       return 'Offensive'
#     else:
#       return 'Neither'
    return predicted_class[0]

## BERT + CNN


In [ ]:
# Define CNN model architecture
def CNN_model():
    # Input layer for text
    text_in = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

    # Preprocessing layer using a BERT model
    preprocess_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_in = preprocess_layer(text_in)

    # BERT encoder layer
    encode = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encode(encoder_in)

    # Extract sequence output from BERT
    network_layer = sequence_output = outputs["sequence_output"]

    # Add CNN layers
    network_layer = tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same')(network_layer)  # CNN Layer 1
    network_layer = tf.keras.layers.MaxPooling1D(pool_size=2)(network_layer)  # MaxPooling Layer 1

    network_layer = tf.keras.layers.Conv1D(128, 3, activation='relu', padding='same')(network_layer)  # CNN Layer 2
    network_layer = tf.keras.layers.MaxPooling1D(pool_size=2)(network_layer)  # MaxPooling Layer 2

    # Add additional CNN + MaxPooling layers
    network_layer = tf.keras.layers.Conv1D(256, 3, activation='relu', padding='same')(network_layer)  # CNN Layer 3
    network_layer = tf.keras.layers.MaxPooling1D(pool_size=2)(network_layer)  # MaxPooling Layer 3

    # Global MaxPooling Layer
    network_layer = tf.keras.layers.GlobalMaxPool1D()(network_layer)

    # Add fully connected layers
    network_layer = tf.keras.layers.Dense(512, activation="relu")(network_layer)  # Dense Layer 1
    network_layer = tf.keras.layers.Dropout(0.3)(network_layer)  # Dropout Layer 1

    network_layer = tf.keras.layers.Dense(256, activation="relu")(network_layer)  # Dense Layer 2
    network_layer = tf.keras.layers.Dropout(0.3)(network_layer)  # Dropout Layer 2

    # Final classification layer
    network_layer = tf.keras.layers.Dense(3, activation="softmax", name='classifier')(network_layer)

    # Create and return the model
    model = tf.keras.Model(inputs=text_in, outputs=network_layer)
    return model

In [ ]:
# Building model
cnn_model = CNN_model()
# Defining loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
cnn_model.summary()

In [ ]:
# Fine-tuning and Training models
epochs_count = 20
steps_each_epoch = tf.data.experimental.cardinality(train_ds).numpy()
number_of_training_steps = steps_each_epoch * epochs_count
number_of_warmup_steps = int(0.1*number_of_training_steps)
init_lr = 2e-5

# Creating_optimiser
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=number_of_training_steps,
                                          num_warmup_steps=number_of_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
# Model Compiling
cnn_model.compile(optimizer=optimizer,
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=tf.keras.metrics.SparseCategoricalAccuracy('accuracy'))

print(f'Training model with {tfhub_handle_encoder}')

In [ ]:
# Training Model
cnn_history = cnn_model.fit(x=train_ds,
                            validation_data=val_ds,
                            epochs=epochs_count,
                            class_weight=class_weight)

In [ ]:
model_path = os.path.join(save_dir, 'bert_cnn_model.h5')
cnn_model.save(model_path)
print(f"Model saved to {model_path}")

In [ ]:
# Creating graphs for accuracy and loss
# Fining values
history_dict = cnn_history.history
print(history_dict.keys())
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']
epochs = range(1, len(acc) + 1)

# Defining figures
fig = plt.figure(figsize=(12, 10))
fig.tight_layout()
# 1st figure
plt.subplot(2, 1, 1)
plt.plot(epochs, loss, 'r', label='Training-loss')
plt.plot(epochs, val_loss, 'b', label='Validation-loss')
plt.title('Training & validation Loss')
plt.xlabel('Epochs-Count')
plt.ylabel('Loss')
plt.legend()
# 2nd Figure
plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training-accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation-accuracy')
plt.title('Training & Validation Accuracy')
plt.xlabel('Epochs-Count')
plt.ylabel('Acc')
plt.legend(loc='lower right')

In [ ]:
# Predicting on test dataset
text = []
test_labels = []
predictions = []
for text, label in test_ds.take(-1):
  text = text.numpy()
  test_labels.append(label.numpy())
  predictions.append(cnn_model.predict(text))

from itertools import chain
flatten_list = list(chain.from_iterable(predictions))

y_pred = np.argmax(flatten_list, axis=-1)
y_test = np.array(list(chain.from_iterable(test_labels)))


In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Drawing Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1', 'Class 2'], yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Upload models to hugging face

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
# login(token="") # Your HF's token here

In [ ]:
import tensorflow as tf
from huggingface_hub import HfApi, HfFolder, Repository, create_repo, login

In [ ]:
# # Tên repository của bạn trên Hugging Face
# repo_name_lstm_and_cnn = "bert_lstm_and_bert_cnn_model"  # Đặt tên repo
# create_repo(repo_name_lstm_and_cnn, exist_ok=True)

In [ ]:
from huggingface_hub import upload_folder

In [ ]:
# Đường dẫn thư mục mô hình BERT + LSTM và BERT + CNN
model_path_lstm_and_cnn = "/gdrive/MyDrive/CS431/model"

# Upload mô hình lên Hugging Face
upload_folder(
    folder_path=model_path_lstm_and_cnn,
    # repo_id="VinhKHMT/bert_lstm_model",  # Thay thế với tên repo của bạn
    repo_id="DinhVu/bert_lstm_and_bert_cnn_model",
    commit_message="Initial commit BERT + LSTM and BERT + LSTM + BERT-based",
)

# Demo


In [ ]:
from official.nlp.optimization import AdamWeightDecay, WarmUp

with tf.keras.utils.custom_object_scope({
    'KerasLayer': hub.KerasLayer,
    'AdamWeightDecay': AdamWeightDecay,
    'WarmUp': WarmUp
}):
    cnn_model = tf.keras.models.load_model('/gdrive/MyDrive/CS431/model/bert_lstm_model_3.h5')

In [ ]:
from official.nlp.optimization import AdamWeightDecay, WarmUp

with tf.keras.utils.custom_object_scope({
    'KerasLayer': hub.KerasLayer,
    'AdamWeightDecay': AdamWeightDecay,
    'WarmUp': WarmUp
}):
    lstm_model = tf.keras.models.load_model('/gdrive/MyDrive/CS431/model/bert_lstm_model.h5')


In [ ]:
def predict_lstm(text):
    prediction = lstm_model.predict([text])
    predicted_class = np.argmax(prediction, axis=-1)
#     if predicted_class == '0':
#       return 'Hate'
#     elif predicted_class == '1':
#       return 'Offensive'
#     else:
#       return 'Neither'
    return predicted_class[0]

In [ ]:
def predict_cnn(text):
    prediction = lstm_model.predict([text])
    predicted_class = np.argmax(prediction, axis=-1)
#     if predicted_class == '0':
#       return 'Hate'
#     elif predicted_class == '1':
#       return 'Offensive'
#     else:
#       return 'Neither'
    return predicted_class[0]

In [ ]:
q# Giao diện cho mô hình CNN
iface_cnn = gr.Interface(
    fn=predict_cnn,  # Hàm dự đoán CNN
    inputs="text",  # Input là văn bản
    outputs="text",  # Output hiển thị lớp dự đoán
    title="BERT + CNN Model",
    description="Dự đoán lớp văn bản (0: Hate, 1: Offensive, 2: Neither) sử dụng mô hình BERT + CNN.",
    live=True
)

# Giao diện cho mô hình LSTM
iface_lstm = gr.Interface(
    fn=predict_lstm,  # Hàm dự đoán LSTM
    inputs="text",  # Input là văn bản
    outputs="text",  # Output hiển thị lớp dự đoán
    title="BERT + LSTM Model",
    description="Dự đoán lớp văn bản (0: Hate, 1: Offensive, 2: Neither) sử dụng mô hình BERT + LSTM.",
    live=True
)

# Chạy cả hai giao diện riêng biệt
iface_cnn.launch()
iface_lstm.launch()
